In [1]:
import pandas as pd
import numpy as np
from numpy import *
from numpy.linalg import multi_dot

from scipy.stats import norm
from tabulate import tabulate

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize']=[24.0,8.0]
matplotlib.rcParams['font.size']=10
matplotlib.rcParams['lines.linewidth']=2.0

import cufflinks as cf
cf.set_config_file(offline=True,dimensions=((1000,600)))

import plotly.express as px
px.defaults.template, px.defaults.width, px.defaults.height = "plotly_white", 1000, 600


In [2]:
asset_dist=pd.DataFrame({'Asset':['A','B','C','D'],'meu':[0.04,0.08,0.12,0.15],'sigma':[0.07,0.12,0.18,0.26]}).set_index('Asset')
asset_dist

meu  sigma
Asset             
A      0.04   0.07
B      0.08   0.12
C      0.12   0.18
D      0.15   0.26

In [3]:
symbol=list(asset_dist.index)
symbol

['A', 'B', 'C', 'D']

In [4]:
num_of_assets=asset_dist.shape[0]

In [5]:
rho=np.array([[1,0.2,0.5,0.3,0.2,1,0.7,0.4,0.5,0.7,1,0.9,0.3,0.4,0.9,1]]).reshape(4,4)
rho

array([[1. , 0.2, 0.5, 0.3],
       [0.2, 1. , 0.7, 0.4],
       [0.5, 0.7, 1. , 0.9],
       [0.3, 0.4, 0.9, 1. ]])

In [6]:
# np.matrix([[1,0.2,0.5,0.3],[0.2,1,0.7,0.4],[0.5,0.7,1,0.9],[0.3,0.4,0.9,1]])

# alternate way to create matrix in numpy

In [7]:
w=np.array(num_of_assets*[1/num_of_assets])[:,newaxis]
w

array([[0.25],
       [0.25],
       [0.25],
       [0.25]])

In [8]:
w /= sum(w)

In [9]:
ret=np.array(asset_dist['meu'])[:,newaxis]
ret

array([[0.04],
       [0.08],
       [0.12],
       [0.15]])

In [10]:
sigma=np.array(asset_dist['sigma']).flatten()
sigma


array([0.07, 0.12, 0.18, 0.26])

In [11]:
w.T@ret

array([[0.0975]])

In [12]:
# To create identity matrix
# np.identity(4)

In [13]:
s=np.diag(sigma)
s

array([[0.07, 0.  , 0.  , 0.  ],
       [0.  , 0.12, 0.  , 0.  ],
       [0.  , 0.  , 0.18, 0.  ],
       [0.  , 0.  , 0.  , 0.26]])

In [14]:
port_cov=multi_dot([s,rho,s])
port_cov

array([[0.0049 , 0.00168, 0.0063 , 0.00546],
       [0.00168, 0.0144 , 0.01512, 0.01248],
       [0.0063 , 0.01512, 0.0324 , 0.04212],
       [0.00546, 0.01248, 0.04212, 0.0676 ]])

# B.2

# Explain in plain English what this optimiztion does

This optimization selects a portfolio of given assets based on the minimization of risk (portfolio variance) subject to a return constraint of 10%. 

In [15]:
import scipy.optimize as sco

In [16]:
initial_weights=num_of_assets*([1/num_of_assets])
initial_weights

[0.25, 0.25, 0.25, 0.25]

In [17]:
def portfolio_stats(weights):
    
    weights=np.array(weights)[:,newaxis]
    port_rets=weights.T@ret
    port_cov=multi_dot([s,rho,s])
    port_vol=np.sqrt(multi_dot([weights.T,port_cov,weights]))
    
    return np.array([port_rets, port_vol]).flatten()

In [18]:
portfolio_stats(initial_weights)[0]

0.0975

In [19]:
import scipy.optimize as sco
    

In [20]:
#Minimizing Variance

def min_variance(weights):
    
    return portfolio_stats(weights)[1]**2

In [21]:
min_variance(initial_weights)

0.017851250000000006

In [22]:
bunds=tuple((-1,1) for x in range(num_of_assets))

In [23]:
#Specify constraints
target_ret=0.1

cons=({'type':'eq','fun': lambda x: sum(x)-1},
     {'type':'eq','fun': lambda x: portfolio_stats(x)[0]-target_ret})

In [24]:
opt_variance = sco.minimize(min_variance,
                   initial_weights,
                   method='SLSQP',
                   bounds=bunds,
                   constraints=cons)
                  
opt_variance      

     fun: 0.01754523151198075
     jac: array([0.00457919, 0.02492006, 0.04526087, 0.06051648])
 message: 'Optimization terminated successfully'
    nfev: 55
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([ 0.05867619,  0.75902844, -0.31954573,  0.50184109])

In [25]:
std_optimal_port=np.sqrt(opt_variance['fun'])*100
std_optimal_port

13.245841427399299

In [26]:
list(zip(asset_dist.index,np.round(opt_variance['x'],3)))


[('A', 0.059), ('B', 0.759), ('C', -0.32), ('D', 0.502)]

In [27]:
optimal_weights=np.round(opt_variance['x'],3).flatten()[:,newaxis]
optimal_weights

array([[ 0.059],
       [ 0.759],
       [-0.32 ],
       [ 0.502]])

In [28]:
#Efficient frontier parameters
target_rets=linspace(0.02,0.2,100)
target_vol=[]

for tr in target_rets:
    
    ef_cons=({'type':'eq','fun': lambda x: sum(x)-1},
     {'type':'eq','fun': lambda x: portfolio_stats(x)[0]-tr})
    
    opt_ef=sco.minimize(min_variance,
                   initial_weights,
                   method='SLSQP',
                   bounds=bunds,
                   constraints=ef_cons)
    
    target_vol.append(np.sqrt(opt_ef['fun']))

In [29]:
#Dataframe for Efficient Frontier

efport=pd.DataFrame({'target_rets':np.array(target_rets)*100,
                    'target_vols':np.array(target_vol)*100})

efport.head()

target_rets  target_vols
0     2.000000     5.010601
1     2.181818     4.571351
2     2.363636     4.178280
3     2.545455     3.845573
4     2.727273     3.590053

In [30]:
# Plot efficient frontier portfolio

fig = px.scatter(efport,
                 x='target_vols',
                 y='target_rets',
                 color='target_rets',
                 labels={'target_rets':'Expected Return %',
                        'target_vols': 'Expected Volatility %'},
                 title="Efficient Frontier Portfolio").update_traces(mode='markers',
                                                                    marker=dict(symbol='cross'))

#plot target return portfolio

fig.add_scatter(mode='markers',
                x=[np.sqrt(opt_variance['fun'])*100],
                y=np.array([target_ret])*100,
                marker=dict(color='red',size=20, symbol='star'),
                name = 'Target Return Portfolio'
               ).update(layout_showlegend=False)



fig.show()

# B.3

In [31]:
#Specify constraints

cons=({'type':'eq','fun': lambda x: sum(x)-1})

In [32]:
global_min_variance = sco.minimize(min_variance,
                   initial_weights,
                   method='SLSQP',
                   bounds=bunds,
                   constraints=cons)
                  
global_min_variance     

     fun: 0.0009213360235621635
     jac: array([0.0032245 , 0.00317659, 0.00456461, 0.00320273])
 message: 'Optimization terminated successfully'
    nfev: 65
     nit: 13
    njev: 13
  status: 0
 success: True
       x: array([ 0.88226808,  0.66499454, -1.        ,  0.45273738])

In [33]:
global_min_port_vol= np.sqrt(global_min_variance['fun'])*100
global_min_port_vol

3.0353517482528503

In [34]:
global_min_port_ret=global_min_variance['x'].T@ret.flatten()
global_min_port_ret

0.0364008937277539

In [35]:
#plot minimum variance portfolio

fig.add_scatter(mode='markers',
                x=[np.sqrt(global_min_variance['fun'])*100],
                y=np.array([global_min_port_ret])*100,
                marker=dict(color='green',size=20, symbol='star'),
                text = 'Min Variance Portfolio'
               ).update_traces(textposition="bottom right")
fig.show()
